# Dummy EAS reconstruction

This is a simple example on how GRANDlib classes can be used for data I/O in the case of shower parameters reconstruction. The traces used do not represent any real EAS traces, and the reconstruction used is completely meaningless, serving only ilustrational purpose.

First, let's generate a ROOT file with random, dummy events. The old file will be cleared if exists. This uses an example script examples/io/DataStoringExample.py. Please be sure to restart kernel each time you try this script.


In [1]:
import os.path

if os.path.isfile("dummy_data.root"):
    os.remove("dummy_data.root")
# Can't use %run - leaves stuff in memory that is needed empty later
#%run ../io/DataStoringExample.py dummy_data.root
os.system("../io/DataStoringExample.py dummy_data.root")

Wrote trun
Wrote tadccounts
Wrote trawvoltage
Wrote tvoltage
Wrote tefield
Wrote tshower
Finished writing file dummy_data.root


0

Now we import the necessary python modules

In [2]:
import sys
import os
from grand.grandlib_classes.grandlib_classes import *
import grand.grandlib_classes.grandlib_classes as g

Let's create an Event, and provide the file name, run and event numbers (which we know to exist) for it

In [3]:
e = Event()
e.file = "dummy_data.root"
e.run_number = 0
e.event_number = 0

The Event has to be told to initialise its contents from the provided file, with provided run and event numbers.
We tell the Event that the tshower it reads from the file contains simulated (not reconstructed) values.

In [4]:
e.fill_event_from_trees(simshower=True)

Just for checking - what are the contents of the beginning of the first trace X of the Efield?

In [11]:
print(e.efields[0].trace_x[:10])

[-0.000257080071605742, 0.001928100478835404, 0.00038562010740861297, -0.000257080071605742, 0.001671020407229662, -0.001028320286422968, 0.002185180550441146, 0.0023137207608669996, -0.0008997802506200969, 0.00141394033562392]


Initialise the reconstruction shower in the Event:

In [6]:
e.shower = Shower()

Storing the totally meaningless energy "reconstruction" in the Event:

In [7]:
e.shower.energy_em = 10**21*(np.max([np.max(np.sqrt(np.array(ef.trace_x)**2+np.array(ef.trace_y)**2+np.array(ef.trace_z)**2)) for ef in e.efields]))
e.shower.energy_primary = 1.2*e.shower.energy_em

Storing the totally meaningless Xmax "reconstruction" in the Event:

In [8]:
e.shower.Xmax = 1e6*(np.max([np.max(np.sqrt(np.array(ef.trace_x)**2+np.array(ef.trace_y)**2+np.array(ef.trace_z)**2)) for ef in e.efields]))

e.shower.Xmaxpos.x = 1e5*np.sum([np.sum(ef.trace_x) for ef in e.efields])
e.shower.Xmaxpos.y = 1e5*np.sum([np.sum(ef.trace_y) for ef in e.efields])
e.shower.Xmaxpos.z = 1e5*np.sum([np.sum(ef.trace_z) for ef in e.efields])

Storing the totally meaningless angle reconstruction in the Event:

In [9]:
e.shower.zenith = np.arccos(e.shower.Xmaxpos.z/np.sqrt(e.shower.Xmaxpos.x**2+e.shower.Xmaxpos.y**2+e.shower.Xmaxpos.z**2))
e.shower.azimuth = np.sign(e.shower.Xmaxpos.y)*np.arccos(e.shower.Xmaxpos.x/np.sqrt(e.shower.Xmaxpos.x**2+e.shower.Xmaxpos.y**2))

Write the reconstructed shower to another file:

In [10]:
if os.path.isfile("shower_reco.root"):
    os.remove("shower_reco.root")

e.write_shower("shower_reco.root")

No valid tshower TTree in the file shower_reco.root. Creating a new one.
